# Displacement vectors search

![Task](pics/task.jpg)

In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import cv2 as cv
import numpy as np
import math
import pandas as pd
%matplotlib inline
def sns_set(with_grid = False):
    sns.set()
    sns.set_style("whitegrid", {'axes.grid' : with_grid})

In [2]:
def sad(a, b):
    return np.mean(np.abs(a.astype(np.float64) - b.astype(np.float64)))